In [6]:
import pandas as pd
import json
import data.utils as data_utils


# Read the JSON file
with open("concept_files/cub_per_class.json", 'r') as file:
    data = json.load(file)

# Create lists for each row
rows = []

# Iterate through the dictionary and create a row for each concept
for species, concepts in data.items():
    # Handle both single concepts and lists of concepts
    if isinstance(concepts, list):
        for concept in concepts:
            rows.append({'species': species, 'concept': data_utils.format_concept(concept)})
    else:
        rows.append({'species': species, 'concept': data_utils.format_concept(concepts)})

# Create DataFrame from the rows
df = pd.DataFrame(rows)

df


,species,concept
0,Black footed Albatross,black feet
1,Black footed Albatross,dark wingtips
2,Black footed Albatross,large size
3,Black footed Albatross,large wingspan
4,Black footed Albatross,long wings
...,...,...
1681,Common Yellowthroat,greenish yellow back and wings
1682,Common Yellowthroat,olive green back
1683,Common Yellowthroat,white belly
1684,Common Yellowthroat,yellow throat and breast


In [7]:
df['concept'].value_counts()

concept
small body                    64
white belly                   55
long tail                     36
yellow eyes                   28
black bill                    24
                              ..
blue sheen on the wings        1
short dark legs                1
brown back and white belly     1
black mask on the face         1
yellow head and breast         1
Name: count, Length: 685, dtype: int64

In [9]:
def show_bird(df, target_species):
    """
    Display all data for a specific bird species
    
    Args:
        target_species (str): Name of the species to search for
    
    Returns:
        pandas.DataFrame: Filtered DataFrame containing only rows for the specified species
    """
    # Create mask for exact species match
    mask = df['species'] == target_species
    
    # Filter DataFrame and return result
    bird_data = df[mask]
    
    # Return empty DataFrame with message if no matches found
    if len(bird_data) == 0:
        print(f"No data found for species: {target_species}")
        return pd.DataFrame()
    # display(bird_data)  
    return bird_data

def show_concepts(df, keywords, neg=False):
    """
    Filter DataFrame rows where concepts contain any of the given keywords (case-insensitive)
    
    Args:
        keywords (str or list): Single keyword or list of keywords to search for in the concepts column
        
    Returns:
        pandas.DataFrame: Filtered DataFrame containing rows where concepts contain any of the keywords
    """
    # Convert single string input to list
    if isinstance(keywords, str):
        keywords = [keywords]
    
    # Convert all keywords to lowercase
    keywords = [k.lower() for k in keywords]
    
    # Create mask for matching any keyword
    mask = df['concept'].str.lower().apply(lambda x: any(k in x for k in keywords))
    
    if not neg:
        matching_rows = df[mask]
    else:
        matching_rows = df[~mask]
    
    # Return empty DataFrame if no matches found
    if matching_rows.empty:
        print(f"No concepts found containing any of these keywords: {keywords}")
        return pd.DataFrame()
    
    return matching_rows

In [15]:
# Replace 'output.csv' with your desired filename
df.to_csv('output.csv', index=False)

In [11]:
show_concepts(df, " and ")

,species,concept
36,Crested Auklet,black back and wings
40,Crested Auklet,white breast and belly
41,Crested Auklet,black legs and feet
45,Least Auklet,black cap and back
46,Least Auklet,black head and back
...,...,...
1659,Rock Wren,brown and gray plumage
1663,Rock Wren,shy and secretive behavior
1680,Common Yellowthroat,yellow throat and breast
1681,Common Yellowthroat,greenish yellow back and wings


In [12]:
import pandas as pd

def split_concepts(df):
    # Create an empty list to store new rows
    new_rows = []
    
    for _, row in df.iterrows():
        species = row['species']
        concept = row['concept']
        
        if ' and ' in concept:
            # Split the concept by 'and'
            parts = concept.split(' and ')
            
            # Handle the first part (it's complete as is)
            new_rows.append({
                'species': species,
                'concept': parts[0]
            })
            
            # Handle the second part
            # If it's a single word, add the adjectives from the first part
            second_part = parts[1].strip()
            if len(second_part.split()) == 1:
                # Get adjectives from first part (all words except the last)
                first_part_words = parts[0].split()
                adjectives = ' '.join(first_part_words[:-1])
                second_part = f"{adjectives} {second_part}"
            
            new_rows.append({
                'species': species,
                'concept': second_part
            })
        # else:
        #     # If no 'and' in concept, keep the row as is
        #     new_rows.append({
        #         'species': species,
        #         'concept': concept
        #     })
    
    # Create new dataframe from the processed rows
    return pd.DataFrame(new_rows)

# Example usage:
result = split_concepts(df)

In [14]:
n=0
result[n*50:(n+1)*50]

,species,concept
0,Crested Auklet,black back
1,Crested Auklet,black wings
2,Crested Auklet,white breast
3,Crested Auklet,white belly
4,Crested Auklet,black legs
5,Crested Auklet,black feet
6,Least Auklet,black cap
7,Least Auklet,black back
8,Least Auklet,black head
9,Least Auklet,black back
